In [1]:
from scripts.dataset_walker import DatasetWalker
from scripts.knowledge_reader import KnowledgeReader
import json

In [2]:
ds_walker = DatasetWalker('train', './data', labels=True)
kng_reader = KnowledgeReader(dataroot='./data/')
kng_reader.get_entity_name('hotel', 11)

'BRIDGE GUEST HOUSE'

In [18]:
import re
RE_PUNC = re.compile(r'[!"#$%&()*+,-./:;<=>?@\[\]\\^`{|}~_\']')
def remove_punc(_text):
    return RE_PUNC.sub(" ", _text)

In [25]:
train_data = []
history_max_utterances = 400
for m, (log, label) in enumerate(ds_walker):

    #print("m", m)

    if(label['target'] == False):
        continue

    with open('./baseline/resources/entity_mapping.json', 'r') as fr:
        all_entity_names_norm = json.load(fr)
    
    curr_history = ""
    curr_sample = {}

    for utter in log:
        curr_history += utter['text'] + " "
    
    curr_history = remove_punc(curr_history)
    curr_history = curr_history[-history_max_utterances:]

    curr_sent_words = curr_history.split(" ")
    entity_name = kng_reader.get_entity_name(label['knowledge'][0]['domain'], label['knowledge'][0]['entity_id'])
    #entity_name_words = entity_name.split(" ")
    entity_names = [entity_name] + all_entity_names_norm.get(entity_name.lower(), [])  ## get all the names in the entity

    #print(entity_names)

    label = ["O"]*len(curr_sent_words)

    for i,word in enumerate(curr_sent_words):

        for e_name in entity_names:
            entity_name_words = e_name.split(" ")
            if (word.lower() == entity_name_words[0].lower()):
                temp = i+1
                flag = True
                for j in entity_name_words[1:]:
                    if (temp >= len(curr_sent_words) or curr_sent_words[temp].lower() != j.lower()):
                        flag = False
                        break
                    temp += 1
                
                if flag:
                    for k in range(len(entity_name_words)):
                        if (k == 0):
                            label[i] = "B-ent"
                        else:
                            label[i+k] = "I-ent"
                    
                    break
    
    # if(not "B-ent" in label):
    #     print("m",m)
    #     print(entity_names)

    curr_sample['sentence'] = curr_history
    curr_sample["label"] = label

    train_data.append(curr_sample)


with open('train.json', 'w') as f:
    json.dump(train_data, f, indent=4)     